### Instalar requerimientos

In [5]:
%pip install requests
%pip install biopython
%pip install py3Dmol
%pip install stmol
%pip install biotite
%pip install ipython_genutils

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Using cached biotite-0.41.1.tar.gz (17.4 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... \

In [1]:
from Bio import SeqIO

def parse_fasta_file(fasta_file) -> dict:
    protein_dict = {}
    
    with open(fasta_file, "r") as handle:
        for record in SeqIO.parse(handle, "fasta"):
            protein_name = record.id
            protein_sequence = str(record.seq)
            protein_dict[protein_name] = protein_sequence
    
    return protein_dict

fasta_file = "data/HTRA2_WT.fasta"

protein_dict = parse_fasta_file(fasta_file)

In [2]:
import requests

ESMFOLD_URL = "https://api.esmatlas.com/foldSequence/v1/pdb/"

def fold_sequence(fastq_seq:str, output_pdb_filepath:str) -> dict:
    results = {}
    if len(fastq_seq) >= 400:
        raise("Error: la longitud de data debe ser menor a 400 caracteres.")
    else:
        try:
            response = requests.post(ESMFOLD_URL, data=fastq_seq, verify=False)
            response.raise_for_status()  # Esto lanzará una excepción si la solicitud no fue exitosa
            results['status_code'] = response.status_code
            with open(output_pdb_filepath, "w") as pdb_file:
                pdb_file.write(response.text)
            results['pdb_file_path'] = output_pdb_filepath
            return results
        except requests.exceptions.RequestException as e:
            print(f"Ocurrió un error al hacer la solicitud: {e}")



print(fold_sequence(protein_dict['HTRA2_HUMAN_madura'], 'results/HTRA2_HUMAN_madura.pdb'))

/opt/homebrew/lib/python3.11/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.esmatlas.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


{'status_code': 200, 'pdb_file_path': 'results/HTRA2_HUMAN_madura.pdb'}


In [3]:
import py3Dmol
from stmol import showmol
import biotite.structure.io as bsio
import biotite.structure as bs

def render_mol(pdb):
    pdbview = py3Dmol.view()
    pdbview.addModel(pdb, 'pdb')
    pdbview.setStyle({'cartoon': {'color': 'spectrum'}})
    pdbview.setBackgroundColor('white')
    pdbview.zoomTo()
    pdbview.zoom(1, 800)
    pdbview.spin(False)
    pdbview.addSurface(py3Dmol.VDW, {'opacity': 0.5})
    showmol(pdbview, height=500, width=660)

# Define el nombre del archivo PDB
pdb_name = "your_pdb_file"  # Reemplaza con el nombre de tu archivo PDB sin la extensión

# Leer el archivo PDB
pdb_path = f"results/HTRA2_HUMAN_madura.pdb"
with open(pdb_path, "r") as archivo:
    pdb_string = archivo.read()

# Cargar la estructura y calcular los valores necesarios
struct = bsio.load_structure(pdb_path, extra_fields=["b_factor"])
b_value = round(struct.b_factor.mean(), 4)
centroid = round(bs.centroid(struct).mean(), 4)
gyration = round(bs.gyration_radius(struct).mean(), 4)

# Mostrar los valores calculados
print(f"B-factor medio: {b_value}")
print(f"Centroid: {centroid}")
print(f"Gyration radius: {gyration}")

# Renderizar la molécula
render_mol(pdb_string)


ModuleNotFoundError: No module named 'ipython_genutils'